# Recurrent Neural Networks (RNNs)

Here I will be summarizing and reviewing chapters 8 and 9 of the online textbook *Dive Into Deep Learning*. I will not be covering every single subsection in detail, but will instead focus on the core ideas of RNNs and their implmentation. This means I will be avoiding the authors focus on language models, which is their motivating example throughout.

## The Basics

In image recognition (and related tasks) CNNs use convolutional layers to effectively leverage the spatial structure of the data. This begs the question of wether we might develop other speacialized systems to deal with data that has a different structure? The answer is yes, and in this case the answer is Recurrent Neural Networks (RNNs) which leverage the sequential and temporal relationships of certain data. It is pretty clear to see that this type of network can have applications to a wide variety of topics. To name a few general areas: video, audio, text, and any kind of time series.

A RNN is not wildily different than a CNN or dense network. The only differece (at least on a surface level) is that RNNs use past information to inform future outputs. If we are looking at some piece of data $x_t$ at time $t\in\mathbb{N}$ then we want to predict...

$$ x_t\sim p(x_t|x_{t-1},\ldots,x_{1}) $$

This presents a problem, namely, the size of the input increases as time does. There are two main approaches to handling this: autoregressive and latent autoregressive models. The former only observes a fixed amount of history (say $x_{t-1}$ through $x_{t-\tau}$ for some $\tau$), while the latter compiles the history into a hidden state ($h_t=g(h_{t-1},x_{t-1})$). We will be focusing on the latent autoregressive models as that is how RNNs are implemented in modern deep learning.

In a neural network we can write the relationship between an input ($\mathbf{x}_t$) and the hidden state ($\mathbf{h}_t$) as follows:

$$ \mathbf{h}_t = \sigma(\mathbf{W}_{x}\mathbf{x}_t + \mathbf{W}_{h}\mathbf{h}_{t-1} + \mathbf{b}_h) $$

We note that we now have two sets of weights to describe how to use the previous hidden state and the current input. We also note that this may not be the only way to describe the structure of a recurrent layer.

Just like with other network structures we can train RNNs by using backpropogation, but because of the hidden state and its dependence on previous inputs we must now backpropogate through time. To show the structure of the gradients we can examine a simplified model with the hidden state $h_t$ and the output $o_t$.

$$
\begin{align}
&h_t=f(x_t,h_{t-1},w_h)\\
&o_t=g(h_t,w_o)
\end{align}
$$

The goal is to produce this sequence of $T$ outputs ($o_t$) with true labels $y_t$, so our total loss for some loss function $l$ is given as follows:

$$ L = \frac{1}{T}\sum_{t=1}^{T}l(y_t,o_t) $$

We then want to take the derivative of this with respect to the parameter we are trying to update ($w_h$).

$$ \frac{\partial L}{\partial w_h} = \frac{1}{T}\sum_{t=1}^{T}\frac{\partial l(y_t,o_t)}{\partial w_h} $$

Using the chain rule and expanding the summand we have...

$$ \frac{\partial l(y_t,o_t)}{\partial w_h} = \frac{\partial l(y_t,o_t)}{\partial o_t}\cdot\frac{\partial g(h_t,w_o)}{\partial h_t}\cdot\frac{\partial h_t}{\partial w_h} $$

The last partial derivative is what needs to be unfolded in time as $h_{t-1}$ also depends on $w_h$. Doing so gives us the following expression:

$$ \frac{\partial h_t}{\partial w_h} = \frac{\partial f(x_t,h_{t-1},w_h)}{\partial w_h} + \sum_{i=1}^{t-1}\Big(\prod_{j=i+1}^t\frac{\partial h_j}{\partial h_{j-1}}\Big)\frac{\partial h_i}{\partial w_h} $$

Clearly this computation can become quite costly as we progress in time, and it can also present a situation for gradient explosion. There are a few ways to deal with this, but by far the most common is to truncate after a fixed number of steps. This limits the computation, and is also beneficial because it puts a larger focus on temporaly local data and can result in some amount of regularization.

## Moden RNNs

Our discussion so far serves as a great base for the practical use of RNNs in modern deep learning, but we need to go beyond this basic structure. We will begin with two more complex instantiations of RNNs that both use gating to avoid some of the problems associated with gradient decay/explosion, and to enhance the focus on varying temporal dependencies. The two layers in question are Gated Recurrent Units (GRUs) and Long Short Term Memory (LSTM). The layers are quite similar in design and philosophy, with GRUs actually being a simplified version of LSTM, but we will begin by looking at the former. Both layer types are informed by three observations from RNN type problems. First, data observed much earlier than the current time may still have a large impact on the current output. Instead of accounting for this in the gradient we would like to maintain some form of memory inside the layer. Second, intermediary data may be extremely unimportant, so we would like to skip this inconsequential information inside the hidden state. Finally, the type of data RNNs can be applied to can often exhibit natural breakpoints between related information. A prototypical example would be paragraph breaks or chapter breaks in text data. Because of this we would like to enable our layer with the ability to reset itself to the baseline hidden state. 

In Gated Recurrent Units (GRUs) we have reset and update gates. Each gate is a learnable fully connected layer with a sigmoid activation (to maintain output in $[0,1]$) whose output is used to inform the update to the hidden state. The goal of the reset gate is to control the use of the previous hidden state and is acting on the first and third obvservations discussed above. The goal of the update gate is to control how much we alter the old hidden state based on the new information -- which adresses all of the observations from above. 

The output from the reset gate is applied to the old hidden state in a pointwise multiplication (essentially a masking operation) which is then combined with the input information in a dense layer to obtain a candidate hidden state for the current time. If the output of the reset is mostly 0 then we are dumping the old hidden state in favour of new information, and we can see how it extends from here. Lastly we need to combine the output of the update gate ($\mathbf{z}_t$) with our candidate hiden state ($\tilde{\mathbf{h}}_t$) which is done according to the following formula:

$$ \mathbf{h}_t=\mathbf{h}_{t-1}\odot\mathbf{z}_t + \tilde{\mathbf{h}}_t\odot(\mathbf{1}-\mathbf{z}_t) $$

Essentially the update gate weights how much of the candidate hidden state and the old hidden state we want to keep. An update gate output of 1 implies we want to maintain the hidden state and an update of 0 implies we want to only keep the update. As the authors note, the update gate acts in a long term plasticity function and the reset gate acts in a short term plasticity function. The update gate is controlling whether we update the hidden state at all while the reset gate controles what that update will be.

In Long Short Term Memory (LSTM) we have three gates instead of two. The input gate, forget gate, and output gate are all the output of fully connected layers with sigmoid activations acting on the previous hidden state and the current inoput. Furthermore, there is a fourth dense layer with a $\tanh$ activation whose output serves as the candidate memory to be stored in the LSTM cell. The parameters for all four of these interior layers are learned during training. This cell memory is different than the previous hidden state and is updated using the forget gate ($\mathbf{f}_t$), the input gate ($\mathbf{i}_t$), and the candidate memory ($\tilde{\mathbf{c}}_t$) as follows:

$$ \mathbf{c}_t = \mathbf{f}_t\odot\mathbf{c}_{t-1} + \tilde{\mathbf{c}}_t\odot\mathbf{i}_t $$

Finally, the updated memory is passed through a $\tanh$ activation and combined in a pointwise multiplication operation with the output gate information ($\mathbf{o}_t$) -- this is the updated hidden state ($\mathbf{h}_t$).

$$  \mathbf{h}_t = \mathbf{o}_t\odot\tanh(\mathbf{c}_t) $$

We can see that the process is a little more broken down in the LSTM as opposed to the GRU. Depending on which gates are open and closed certain memory is retained or forgotten, new memory is added or not, and the updated memory is passed on or kept inside the cell. These operations thus satisify the goal of the observations described above. The forget gate controls how much of the old memory is retained, the input gate controls how much of the new data is incorporated, and the output gate controls how much of the cell memory is passed on.

Given that GRUs and LSTM seem to be performing the same function -- albiet in slightly different ways -- one might ask why choose one over the other? It is pretty clear to see that LSTM has more parameters and more stored information along with a slightly more complicated structure. This will make such a layer more computationally expensive to train and use, but (as the authors note) more capable in execution. Speacilized RNN layers extend further than the two we have described (and we will discuss some of this later), but for now we will move past the layer specifics and into how they can be combined in deep learning.

To me the obvious next step is to insert a number of RNN layers on top of eachother with each hidden unit feeding into the next. As well, they might be combined with other deep learning layers in any number of combinations. In practice this is how they are used, but they do tend to be much more fussy than more standard models. Bidirectional RNNs are also another way to utilize deep RNN structures. These work by feeding information from future data backwards into the previous time steps. We will not delve too deep into how this works, but we will note that the most important consideration here is whether that makes sense for the data in question. For example, if you are filling in a blank in a sentence then you have future data that may be useful, but if you are predicting time series data then the whole point is that you don't know the future.

Just like with anything in deep learning, there are endless methods to combine ideas and a large amount of active research. Our discussion of RNNs is certainly not complete, and to that end we will investigate the presence of RNNs in Tensorflow.

In Tensorflow there is support for all of the layers we have discussed (standard RNNs, LSTM, and GRUs). There is also support for bidirectionality, and a new layer we haven't discussed: convolutional LSTM. This layer is esentially the same as a regular LSTM, but uses convolutional networks inside the cell. It is specifically geared towards data such as video where both convolutions and recurrence are applicable.